# Siamese network notes:
 tf.keras.model for siamese network passing input and outputs
 (model(inputs=[inputImage, verificationImage], output = [1,0] - distance)

layer types:
 Layer - allows defining a custom layer

 Conv2d - allows convolution :)

 Dense - fully connected layer

 MaxPooling2d - shrink our data

 Input - base class allows definiton of model inputs

 Flatter - allows to convert convolution into 1d array, which in turn allows to pass data into the Dense form

# Import packages and create folder structure

In [21]:
import tensorflow
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plot
import Configuration.Config as config
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer,Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

for path in config.siamese.values():
    if not os.path.exists(path):
        os.mkdirs(path)
        print("Created directory at: " + path)

# Setting up GPU Growth memory limit

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Getting data - http://vis-www.cs.umass.edu/lfw/
downloading lfw.tgz

# Negative

In [ ]:
# unzip faces in the wild
!cd Siamese && tar -xf lfw.tgz

In [5]:
negativePath = config.siamese['NEGATIVE_PATH']
for dir in os.listdir('Siamese/lfw'):
    for file in os.listdir(os.path.join('Siamese/lfw', dir)):
        existingPath = os.path.join('Siamese/lfw', dir, file)
        newPath = os.path.join(negativePath, file)
        os.replace(existingPath, newPath)

# Positive/Anchor

In [6]:
import PicturePipeline

PicturePipeline.runSiamesePicturePipeline()
# manualy split int onchors (webcam like) and positives

In [9]:
import Configuration.Config as config
import uuid

# from webcam
anchorPath = config.siamese['ANCHOR_PATH']
positivePath = config.siamese['POSITIVE_PATH']
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    #250 x 250
    frame = frame[120:120+250,200:200+250, :]
    
    # a for anchor
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(anchorPath, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # p for positive
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(positivePath, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    cv2.imshow('Anchor/Positives', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
          
cap.release()
cv2.destroyAllWindows()

# Loading/Preprocessing data

In [1]:
import tensorflow as tf
import Configuration.Config as config
anchorPath = config.siamese['ANCHOR_PATH']
positivePath = config.siamese['POSITIVE_PATH']
negativePath = config.siamese['NEGATIVE_PATH']

# this is only for 1 person

anchorData = tf.data.Dataset.list_files(anchorPath+'\*.jpg').take(380)
positiveData = tf.data.Dataset.list_files(positivePath+'\*.jpg').take(380)
negativeData = tf.data.Dataset.list_files(negativePath+'\*.jpg').take(380)

In [2]:
positiveDataset = tf.data.Dataset.zip((anchorData, positiveData, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchorData)))))
negativeDataset = tf.data.Dataset.zip((anchorData, negativeData, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchorData)))))
concatinatedDatasetForAPerson = positiveDataset.concatenate(negativeDataset)

In [27]:
concatinatedDatasetForAPerson = concatinatedDatasetForAPerson.map(tt.preProcessTwinImages)
concatinatedDatasetForAPerson = concatinatedDatasetForAPerson.cache()
concatinatedDatasetForAPerson = concatinatedDatasetForAPerson.shuffle(buffer_size=1024)

# Training partition

In [61]:
trainingImageAmount = round(len(concatinatedDatasetForAPerson) * .7) # 70% of images
trainingData = concatinatedDatasetForAPerson.take(trainingImageAmount)
trainingData = trainingData.batch(16)
trainingData = trainingData.prefetch(8)

# Testing partition

In [65]:
testingData = concatinatedDatasetForAPerson.skip(trainingImageAmount) # taking the other 30 %
testingImageAmount = round(len(concatinatedDatasetForAPerson) * .3)
testingData = testingData.take(testingImageAmount)
testingData = testingData.batch(16)
testingData = testingData.prefetch(8)


# Creation of Model